In [ ]:
import opensrane as opr
import matplotlib.pyplot as plt

### fire_point_source (Physical Effect Model)
#### casal Point Source Fire Radiation

Verification with casal book Example 3.4 

ATTENTION: PARAMETERS ARE SET HOW THE OTHE VALUES BECOME EQUAL TO THE casal EXAMPLE 3.4 HYPOTHESISES

In [ ]:
opr.wipe()

#Define Wind Rose 
windobj=opr.WindData.WindRose(1)
windobj.WindSpeed=0
windobj.WindDirection=90

#Define Site Condition and Geometry
SiteTAg=1
obj=opr.Sites.Site(SiteTAg, Temperature=16+273, Pressure=1*10**5, XSiteBoundary=[0,100,100,0], YSiteBoundary=[0,0,100,100],
                   g=9.81,Humidity=0.79,Airdensity=0.270)

#Define Substance
subsobj=opr.Substance.DataBank.Butene(1) #Use DataBank to Load Material
subsobj.Specific_Heat_of_Combustion=41900*1000

#Define OutFlow Model
OutflowObj=opr.OutFlowModel.TankHole(1, Hole_Diameter=0.05, Hole_Height_FromBot=0, delta_t=500, Cd=1)
#Define Dispersion Model
DispObj=opr.DispersionSpreadModels.LiquidSpread(tag=1, MatTags=[1], OutFlowModelTags=[1],MinDisThickness=0.005,)

#Define Dike for Tanks
opr.Safety.Dike(1,0,3.1415*6**2/4)


PlantObj=opr.PlantUnits.ONGStorage(1,SiteTag=1,DikeTag=1, SubstanceTag=1,    Diameter=10,    
                                   Horizontal_localPosition=0,    Vertical_localPosition=0,
                                   Height=20,SubstanceVolumeRatio=0.9)

#Manually define OutFlow Model for above Unit Object
PlantObj.OutFlowModelObject=OutflowObj
OutflowObj.UnitObject=PlantObj
PlantObj.OutFlowModelObject.Calculate()


#Manually define DispersionSpread Model for above Unit Object
PlantObj.DispersionSpreadModelObject=DispObj
PlantObj.DispersionSpreadModelObject.UnitObject=PlantObj
PlantObj.DispersionSpreadModelObject.Calculate()


obj=opr.PhysicalEffect.fire_point_source(1,minf=0.0501,k=1.5)
obj.UnitObject=PlantObj
[H,Hmax,D,Dprin,alpha,m]=obj._fireGeometry()
[Lp,d,phi,xf,yf]=obj._DistanceToFireCenterGeometry(18,0,1.6)
  

print('H,D,m=',round(H,2),round(D,2),round(m,2))
print('Lp,d,phi=',round(Lp,2),round(d,2),round(phi,2))

print('Thermal Radiation intensity=',obj.Thermal_Radiation_at_Point(18,0,1.6) )

In [ ]:
print('in casal Example 3.4, \nD=6 m \nH=11.5\nPoint coordinate=[18,0,1.6] \nHeat Of Combustion=41900*10^3\nHumidity=0.79\nm=0.05')
print()
print('And the result Thermal Radiation intensity= 2800 W/m^2')
print('That is compatible with the above results!')

### Check the Variation of the Heat Radiation respect to the distance of the point

In [ ]:
# obj.UnitObject.Hlocalcoord=3000
# obj.UnitObject.Vlocalcoord=4000


X=[i*5 for i in range(-100,100)]

I=[]
Lp=[]
DD=[]
Phi=[]
for x in X:

    i=obj.Thermal_Radiation_at_Point(x,0,0)
    I.append(i)


plt.figure(0)
plt.plot(X,I)
plt.xlabel('Horizontal Distance')
plt.ylabel('Thermal Radiation (W/m^2)')




Z=[i*5 for i in range(-100,100)]

I=[]
Lp=[]
DD=[]
Phi=[]
for z in Z:
    i=obj.Thermal_Radiation_at_Point(5,0,z)
    I.append(i)
    
plt.figure(1)
plt.plot(Z,I)
plt.xlabel('Vertical Distance')
plt.ylabel('Thermal Radiation (W/m^2)');